In [18]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import json

In [19]:
urlbase = "https://data.cityofnewyork.us/resource/"

In [20]:
def desde_fecha(fecha_str):
       return f'{fecha_str}T00:00:00.000'

In [21]:
def hasta_fecha(fecha_str):
   return f'{fecha_str}T23:59:59.000'

In [22]:
def extraccion_actual(ini, fin, token):
    url_eventos = f"{urlbase}tvpp-9vvx.json"
    
    param = {
        "$where": f"start_date_time >= '{ini}' AND start_date_time <= '{fin}'",    
    }

    header = {"X-App-Token": token}
    eventos = requests.get(url=url_eventos, params=param, headers=header)
    

    if eventos.status_code != 200:
        print(f"Error Parques: {eventos.text}")
 
    assert eventos.status_code == 200, "Error en la extracción de eventos"
    return eventos.json()




In [23]:
token = os.getenv('NYC_OPEN_DATA_TOKEN')
assert token is not None, "Falta la variable de entorno NYC_OPEN_DATA_TOKEN"


fecha_hoy_str = datetime.now().strftime('%Y-%m-%d')
fecha_actual = desde_fecha(fecha_hoy_str)
fecha_fin = hasta_fecha(fecha_hoy_str)


json_eventos = extraccion_actual(fecha_actual, fecha_fin, token)

In [24]:
df = pd.DataFrame(json_eventos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   event_id             161 non-null    object
 1   event_name           160 non-null    object
 2   start_date_time      161 non-null    object
 3   end_date_time        161 non-null    object
 4   event_agency         161 non-null    object
 5   event_type           161 non-null    object
 6   event_borough        161 non-null    object
 7   event_location       161 non-null    object
 8   street_closure_type  161 non-null    object
 9   community_board      161 non-null    object
 10  police_precinct      161 non-null    object
 11  cemsid               154 non-null    object
 12  event_street_side    3 non-null      object
dtypes: object(13)
memory usage: 16.5+ KB


In [25]:
df

,event_id,event_name,start_date_time,end_date_time,event_agency,event_type,event_borough,event_location,street_closure_type,community_board,police_precinct,cemsid,event_street_side
0,888287,East Green -East 72nd Playground Lawn,2026-02-17T00:00:00.000,2026-02-17T23:59:00.000,Parks Department,Special Event,Manhattan,Central Park: East 72nd Street Playground Lawns,N/A,"8,","19,","2620,",NaN
1,894407,Maintenance,2026-02-17T00:00:00.000,2026-02-17T23:59:00.000,Parks Department,Special Event,Manhattan,Fort Tryon Park: Billings Lawn,N/A,"12,","34,","3994,",NaN
2,871486,Cherry Lawn Closure,2026-02-17T00:00:00.000,2026-02-17T02:00:00.000,Parks Department,Special Event,Manhattan,Madison Square Park: Cherry Lawn,N/A,"5,","13,","11920,",NaN
3,906240,Sol LeWitt Lawn Closure,2026-02-17T00:00:00.000,2026-02-17T14:00:00.000,Parks Department,Special Event,Manhattan,Madison Square Park: Sol Lewitt / 25th St Lawn...,N/A,"5,","13,","257,",NaN
4,889680,Lawn Closure - East 102 St. Landscape A B,2026-02-17T00:00:00.000,2026-02-17T23:59:00.000,Parks Department,Special Event,Manhattan,"Central Park: East 102nd St Landscape A ,Cen...",N/A,"64,","22,","11762, 11761,",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,894797,Soccer - Non Regulation,2026-02-17T20:00:00.000,2026-02-17T22:00:00.000,Parks Department,Sport - Adult,Brooklyn,Sternberg Park (Lindsey Park): Soccer/Football-01,N/A,"01,","90,","9933,",NaN
157,896259,Soccer -Regulation,2026-02-17T20:00:00.000,2026-02-17T22:00:00.000,Parks Department,Sport - Adult,Manhattan,Randall's Island Park: Wards Meadow Fields-Soc...,N/A,"11,","25,","5749,",NaN
158,891683,Soccer - Non Regulation,2026-02-17T20:00:00.000,2026-02-17T22:00:00.000,Parks Department,Sport - Adult,Brooklyn,McCarren Park: Soccer-01,N/A,"01,","94,","5122,",NaN
159,889880,Soccer - Non Regulation,2026-02-17T20:30:00.000,2026-02-17T22:00:00.000,Parks Department,Sport - Adult,Manhattan,Randall's Island Park: Wards Meadow Fields-Soc...,N/A,"11,","25,","5750,",NaN


In [26]:

df['start_date_time'] = pd.to_datetime(df['start_date_time'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
df["start_date_time"] = df["start_date_time"].dt.strftime('%H:%M:%S')
df['end_date_time'] = pd.to_datetime(df['end_date_time'], errors='coerce',  format='%Y-%m-%dT%H:%M:%S.%f')
df["end_date_time"] = df["end_date_time"].dt.strftime('%H:%M:%S')



In [27]:
df.columns
df = df.drop(["event_id", "event_agency", "street_closure_type", 'community_board','police_precinct', 'cemsid',
       'event_street_side' ], axis = 1)


In [28]:

riesgo_map = {
    'Parade': 10,
    'Athletic Race / Tour': 10,
    'Street Event': 8,
    'Special Event': 7,
    'Plaza Event': 6,
    'Plaza Partner Event': 6,
    'Theater Load in and Load Outs': 5,
    'Religious Event': 3,
    'Farmers Market': 2,
    'Sidewalk Sale': 2,
    'Production Event': 1,
    'Sport - Adult': 1,
    'Sport - Youth': 1,
    'Miscellaneous': 1,
    'Open Street Partner Event': 2
}


df['nivel_riesgo_tipo'] = df['event_type'].map(riesgo_map)

In [29]:
df = df.sort_values(by = "nivel_riesgo_tipo", ascending= False)
df = df[df.nivel_riesgo_tipo > 6]

In [30]:
df

,event_name,start_date_time,end_date_time,event_type,event_borough,event_location,nivel_riesgo_tipo
49,Street Health Outreach Wellness Program,08:00:00,17:00:00,Street Event,Brooklyn,HART STREET between BROADWAY and STUYVESANT AV...,8
68,BioBus For PS 310,11:00:00,14:00:00,Street Event,Bronx,BAILEY AVENUE between WEST 193 STREET and WES...,8
2,Cherry Lawn Closure,00:00:00,02:00:00,Special Event,Manhattan,Madison Square Park: Cherry Lawn,7
1,Maintenance,00:00:00,23:59:00,Special Event,Manhattan,Fort Tryon Park: Billings Lawn,7
4,Lawn Closure - East 102 St. Landscape A B,00:00:00,23:59:00,Special Event,Manhattan,"Central Park: East 102nd St Landscape A ,Cen...",7
5,Oval Lawn Closure,00:00:00,12:00:00,Special Event,Manhattan,Madison Square Park: Oval Lawn,7
6,Veterans Lawn Closure,00:00:00,15:00:00,Special Event,Manhattan,Madison Square Park: Veterans Lawn,7
3,Sol LeWitt Lawn Closure,00:00:00,14:00:00,Special Event,Manhattan,Madison Square Park: Sol Lewitt / 25th St Lawn...,7
8,Sol Lewitt Lawn Closure,00:00:00,03:00:00,Special Event,Manhattan,Madison Square Park: Sol Lewitt / 25th St Lawn...,7
9,Veterans Lawn Closure,00:00:00,23:59:00,Special Event,Manhattan,Madison Square Park: Veterans Lawn,7
